In [ ]:
from chemview  import MolecularViewer
from chemview.trajectory import TrajectoryViewer
from chemview.export import display_static

In [ ]:
# We use MDTraj to get a secondary structure
import mdtraj as md
traj = md.load_pdb('2M6K.pdb')
print traj

In [ ]:
top = {}
top['atom_types'] = [a.element.symbol for a in traj.topology.atoms]
top['atom_names'] = [a.name for a in traj.topology.atoms]
top['bonds'] = [(a.index, b.index) for a, b in traj.topology.bonds]
top['secondary_structure'] = md.compute_dssp(traj[0])[0]
top['residue_types'] = [r.name for r in traj.topology.residues ]
top['residue_indices'] = [ [a.index for a in r.atoms] for r in traj.topology.residues ]

mv = MolecularViewer(traj.xyz[0], top)
mv.cylinder_and_strand()
mv

In [ ]:
# Display static
display_static(mv)

In [ ]:
import json

def callback(content):
    with open('data.json', 'wb') as fp:
        json.dump(content['json'], fp)

mv._connect_event('serialize', callback)
mv._remote_call('_handle_serialize')

In [ ]:
%%javascript

var prefix = "https://rawgit.com/gabrielelanaro/chemview/master/chemview/static/";

require.config({
    paths: {
        'jquery': prefix + 'jquery.min',
        'jqueryui': prefix + 'jquery-ui.min',
        'exporter': prefix + 'objexporter.js',
        'three': prefix + 'three.min',
        'base64-arraybuffer': prefix + 'base64-arraybuffer',
        'ArcballControls' : prefix + 'ArcballControls',
        'chemview': prefix + 'chemview',
    },
    shim: {
        three: {
            exports: 'THREE'
        },

        chemview: {
            deps: ['three', 'ArcballControls', 'base64-arraybuffer'],
            exports: 'MolecularViewer'
        },

        ArcballControls: {
            deps: ['three'],
            exports: 'THREE.ArcballControls',
        },
    },
});

Serialize this thing

In [ ]:
%%html

<canvas id="molecular_viewer"></canvas>
<script>

require(['chemview'], function () {
    console.log('done loading');
    var canvas = $("#molecular_viewer").css({width: 400, height: 400});
    var mv = new MolecularViewer(canvas);
    var prefix = "https://rawgit.com/gabrielelanaro/chemview/master/";

    var datafile = prefix + 'docs/source/_static/protein_rendered.json';

    $.getJSON(datafile, function (data){
        mv.deserialize(data);
        mv.animate();

    // Give it a nice zoom
        mv.controls.dollyIn(1.9);
    });

    mv.resize(canvas.width(), canvas.height());
});

</script>
